In [2]:
import numpy as np
import pandas as pd

In [4]:
ds = pd.read_csv("cleaned_data.csv")

In [8]:
X = ds[["company", "name", "year", "kms_driven", "fuel_type"]]
y = ds[["Price"]]

In [24]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe = OneHotEncoder()
ohe.fit(X[["company", "name", "fuel_type"]])

OneHotEncoder()

In [54]:
ct = make_column_transformer((OneHotEncoder(categories = ohe.categories_), 
                              ["company", "name", "fuel_type"]), remainder = 'passthrough', force_int_remainder_cols=False)

In [56]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [58]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(ct, reg)

In [80]:
#training and testing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
scores = []
for i in range(0, 101):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = i)
    pipe.fit(X_train, y_train)
    result = pipe.predict(X_test)
    score = r2_score(y_test, result)
    scores.append(score)

In [88]:
bestindex = np.argmax(scores)

In [90]:
scores[bestindex]

0.9009947984071098

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = bestindex)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Land', 'Mahindra',
       'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan', 'Renault',
       'Sko...
       'i10 Magna 1.2', 'i10 Magna 1.2 Kappa2', 'i10 Sportz',
       'i10 Sportz 1.2', 'i20', 'i20 Active', 'i20 Active 1.2 SX',
       'i20 Active 1.4L SX O', 'i20 Asta', 'i20 Asta 1.2',
       'i20 Asta 1.4 CRDI 6 Speed', 'i20 Magna', 'i20 Magna 1.2',
       'i20 Magna O 1.2', 'i20 Select Variant', 'i20 Sportz 1.2',
       'i20 Sportz 1.4 CRDI'], dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['company', 'name',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [106]:
company = input("Enter company name:")
name = input("Enter car name:")
year = int(input("Enter year:"))
kms_driven = int(input("Enter kms driven:"))
fuel_type = input("Enter fuel type:")
columns = ["company", "name", "year", "kms_driven", "fuel_type"]
myinput = pd.DataFrame(columns = columns, data = [[company, name, year, kms_driven, fuel_type]])
result = pipe.predict(myinput)
print("You should it for ~ price:", round(result[0,0]))

Enter company name: Ford
Enter car name: Figo
Enter year: 2020
Enter kms driven: 30000
Enter fuel type: Petrol


You should it for ~ price: 501111


In [108]:
import pickle as pkl
pkl.dump(pipe, open("CarProject.pkl", "wb"))